# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [2]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [3]:
context = [{
    "role": "system",
    "content": """
You are an AI that generates SQL queries.

Database schema:

CREATE TABLE employees (
    ID_usr INT,
    name VARCHAR,
    department VARCHAR,
    role VARCHAR
);

CREATE TABLE salary (
    ID_usr INT,
    year DATE,
    salary FLOAT
);

CREATE TABLE studies (
    ID INT,
    ID_usr INT,
    educational_level VARCHAR,
    institution VARCHAR,
    years DATE,
    speciality VARCHAR
);
"""
}]


In [4]:
context.append({
    "role": "system",
    "content": """
Examples:

Q: Who is the highest paid employee?
SQL:
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;

Q: What is the average salary per year?
SQL:
SELECT year, AVG(salary)
FROM salary
GROUP BY year;

Q: Which institutions are associated with employees?
SQL:
SELECT DISTINCT institution
FROM studies;
"""
})


In [5]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [6]:
#new
context_user = context.copy()
print(return_CCRMSQL("""YOUR QUERY HERE""", context_user))

I'm ready to help! Please provide me with a question so I can generate the SQL query for you.


In [7]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("YOUR QUERY HERE", old_context_user))

This is your SQL: 

```sql
SELECT * FROM employees;
```

Explanation: This SQL query selects all data from the "employees" table.


In [8]:
#new
print(return_CCRMSQL("YOUR QUERY HERE", context_user))

I'm ready to help! Please provide me with a question so I can generate the SQL query for you.


In [9]:
#old
print(return_CCRMSQL("YOUR QUERY HERE", old_context_user))

This is your SQL:
```sql

Please provide a specific query to assist you.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [13]:
context_user = context.copy()
print(return_CCRMSQL("Which department has the highest average salary?", context_user))


SQL:
SELECT e.department, AVG(s.salary) AS avg_salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
GROUP BY e.department
ORDER BY avg_salary DESC
LIMIT 1;


In [14]:
context_user = context.copy()
print(return_CCRMSQL("Find employees who studied at the same institution", context_user))


SQL:
SELECT e1.name AS employee1, e2.name AS employee2, s.institution
FROM employees e1
JOIN studies s ON e1.ID_usr = s.ID_usr
JOIN employees e2 ON e1.ID_usr != e2.ID_usr
JOIN studies s2 ON e2.ID_usr = s2.ID_usr AND s.institution = s2.institution;


In [15]:
context_user = context.copy()
print(return_CCRMSQL("Which specialization is linked to the highest salaries?", context_user))


SQL:
SELECT st.speciality
FROM studies st
JOIN salary s ON st.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;


## Findings Report (One Page)

### What I tested
- I tested **two prompting approaches** for text-to-SQL:
  - **Old prompt**: long natural-language instructions + table info written in a loose format
  - **New prompt**: explicit `CREATE TABLE` schema + **few-shot examples**
- I then ran **3 additional “creative” prompts** (Exercise section) to see how well the model generalized.

### Results and observations
- **New prompt worked better overall**
  - Output SQL was **more structured** and usually **syntactically correct**
  - The model was more consistent at:
    - selecting the right tables
    - adding JOINs on `ID_usr`
    - using `GROUP BY` and `AVG()` when asked for “average”
- **Few-shot examples improved reliability**
  - The examples acted like “templates”
  - The model copied the style:
    - `JOIN` patterns
    - ordering and limiting results
    - using `DISTINCT` when needed

### Variations that did not work well (hallucinations / wrong logic)
- **Ambiguous requests produced assumptions**
  - When a question required interpretation (for example “specialization linked to highest salaries”), the model sometimes:
    - assumed relationships not explicitly stated
    - selected columns that might not exist
    - mixed up what should be grouped vs. filtered
- **Typical failure patterns**
  - missing a JOIN when it was logically required
  - grouping errors (missing `GROUP BY` while using aggregates)
  - using a field name that looks plausible but is not in the schema

### What I learned
- Prompt clarity has a **direct impact** on SQL correctness.
- Explicit schema in `CREATE TABLE` format reduces hallucination.
- Few-shot examples reduce structural mistakes and improve consistency.
- Ambiguity increases the chance the model invents logic or fields.
- Best practice: **schema + examples + precise question wording** gives the most reliable text-to-SQL output.
